In [12]:
%load_ext autoreload
%autoreload 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
%aimport utils

In [81]:
import concurrent.futures
import requests
import pandas as pd
import numpy as np
from PIL import ImageFile, Image

In [27]:
def getImageSize(uri):
    try:
        r = requests.get(uri, stream=True, timeout=3)
        p = ImageFile.Parser()
        i = 0
        if r.status_code == 200:
            while 1:
                i+=1
                data = r.raw.read(1024)
                if not data:
                    print('not image: uri = ', uri)
                    size = None
                    break
                p.feed(data)
                if p.image:
                    size = p.image.size
                    break
        else:
            print('r.status_code = %d, uri = %s' % (r.status_code, uri))
            size = None
        r.close()
        return size
    except requests.exceptions.Timeout as e:
        print('timeout detected: uri = ', uri)
        print(e)
        return None

In [73]:
try:
    with open('./artworkId2ImageSize.csv') as f:
        df = pd.read_csv(f)
        id2size = {_id:(w,h) for _id,w,h in zip(df.id, df.width, df.height)}
except FileNotFoundError:
    id2size = dict()

In [74]:
len(id2size)

7799

In [75]:
id2url = utils.get_id2url_map()

In [76]:
pending_ids = [_id for _id in id2url if _id not in id2size]
pending_urls = [url for _id, url in id2url.items() if _id not in id2size]

In [77]:
len(pending_ids), len(pending_urls)

(6, 6)

In [78]:
with concurrent.futures.ThreadPoolExecutor(max_workers=40) as executor:
    futures = executor.map(getImageSize, pending_urls)
    for _id, size in zip(pending_ids, futures):
        if size is not None:
            id2size[_id] = size

r.status_code = 404, uri = http://static.ugallery.com/webdata/product/50106/Images/Large_test3.jpg
r.status_code = 404, uri = http://static.ugallery.com/webdata/product/53466/Images/Large_test2.jpg
r.status_code = 404, uri = http://static.ugallery.com/webdata/product/51917/Images/Large_test1.jpg
r.status_code = 404, uri = http://static.ugallery.com/webdata/product/55624/Images/Large_test5.jpg
r.status_code = 404, uri = http://static.ugallery.com/webdata/product/52754/Images/Large_test2.jpg
r.status_code = 404, uri = http://static.ugallery.com/webdata/product/46399/Images/Large_test2.jpg


In [79]:
with open('./artworkId2ImageSize.csv', mode='w') as f:
    data = np.array([(_id, w, h) for _id, (w, h) in id2size.items()])
    df = pd.DataFrame(data=data, columns=('id','width','height'))
    df.to_csv(f, index=False)

In [84]:
ids_with_issues = []

In [93]:
with Image.open("/mnt/workspace/Ugallery/images/%d.jpg" % 43559) as img:
    print(img.size)

(700, 700)


In [85]:
for _id, size in id2size.items():
    with Image.open("/mnt/workspace/Ugallery/images/%d.jpg" % _id) as img:
        try:
            assert size == img.size
        except AssertionError:
            print('id=%d, image size = (%d,%d), validation size = (%d,%d)' % (_id, *img.size, *size))
            ids_with_issues.append(_id)

id=49577, image size = (900,690), validation size = (900,681)
id=49718, image size = (523,532), validation size = (701,700)
id=50798, image size = (549,700), validation size = (561,700)
id=51042, image size = (900,577), validation size = (900,668)
id=52969, image size = (685,700), validation size = (699,700)
id=53153, image size = (710,700), validation size = (182,180)
id=53766, image size = (525,700), validation size = (560,700)
id=50368, image size = (525,700), validation size = (526,700)
id=8823, image size = (720,479), validation size = (270,180)
id=43234, image size = (900,600), validation size = (269,180)
id=43739, image size = (714,700), validation size = (703,700)
id=11518, image size = (624,396), validation size = (284,180)
id=44597, image size = (514,700), validation size = (525,700)
id=45300, image size = (900,446), validation size = (900,449)
id=13034, image size = (687,500), validation size = (248,180)
id=48269, image size = (900,599), validation size = (900,600)
id=48270,